In [1]:
import json

# Update Task Dict
def update_task_dict_from_file(file_name, task_dict):
    with open(file_name, 'r') as f:
        for line in f:
            record = json.loads(line)
            example = {}
            example['context'] = record['context']
            example['category'] = record['category']
            example['question'] = record['question']
            if len(record['answer']['text']) == 1:
                example['answer'] = record['answer']['text'][0]
            else:
                example['answer'] = 'I do not know.'
                
            task_dict['examples'].append(example)
    return task_dict

In [2]:
import json

task_dict_positive={}
task_dict_positive['examples']=[]

task_dict_negative={}
task_dict_negative['examples']=[]

# Generate examples
task_dict_positive = update_task_dict_from_file('C:/Users/ADMIN/Desktop/DATN/Extract_Information/data/aveqa_mave/test/mave_positives.jsonl', task_dict_positive)
task_dict_negative = update_task_dict_from_file('C:/Users/ADMIN/Desktop/DATN/Extract_Information/data/aveqa_mave/test/mave_negatives.jsonl', task_dict_negative)

In [3]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "C:/Users/ADMIN/Desktop/DATN/Extract_Information/baseline/bert_fine_turn/checkpoint"
question_answerer = pipeline("question-answering", model=model_checkpoint)

In [4]:
from tqdm import tqdm

result_dict ={}
result_dict['examples'] =[]

for example in tqdm(task_dict_positive['examples']):
    pred = question_answerer(question=example['question'], context=example['context'])
    example['pred'] = pred['answer']
    example['score'] = pred['score']
    result_dict['examples'].append(example)
    
with open('result_test_aveqa_positive.json', 'w', encoding='utf-8') as f:
    json.dump(result_dict, f, indent=4)
    

100%|██████████| 421/421 [01:07<00:00,  6.22it/s]


In [5]:
min_score_positive = 1
for example in result_dict['examples']:
    min_score_positive = min(min_score_positive, example['score'])

print(min_score_positive)

1.528128018596951e-23


In [6]:
vc = 0 # Correct values when the ground truth has attribute Values (TP)
vw = 0 # Wrong values when the ground truth has attribute Values (FN-FP)

for example in result_dict['examples']:
    if example['answer'] == example['pred']:
        vc += 1
    else:
        vw += 1
print(f"TP: {vc}")
print(f"FP-FN: {vw}")

TP: 417
FP-FN: 4


In [7]:
from tqdm import tqdm

result_dict_negative ={}
result_dict_negative['examples'] =[]

for example in tqdm(task_dict_negative['examples']):
    pred = question_answerer(question=example['question'], context=example['context'])
    example['pred'] = pred['answer']
    example['score'] = pred['score']
    result_dict_negative['examples'].append(example)
    
with open('result_test_aveqa_negative.json', 'w', encoding='utf-8') as f:
    json.dump(result_dict_negative, f, indent=4)

100%|██████████| 141/141 [00:15<00:00,  9.35it/s]


In [8]:
max_score_negative = 0
for example in result_dict_negative['examples']:
    max_score_negative = max(max_score_negative, example['score'])

print(max_score_negative)

0.9890369772911072


In [9]:
nn = 0 # the model can predict No value (I do not know.) when ground truth is No attribute value (TN)
nv = 0 # some incorrect Value when ground truth is No attribute value (FP)

for example in result_dict_negative['examples']:
    if example['score'] < 0.1:
        nn += 1
    else:
        nv += 1
        
print(f"TN: {nn}")
print(f"FP: {nv}")

TN: 140
FP: 1


In [10]:
precision = round((vc / (nv + vc + vw))*100, 2) if (nv + vc + vw) > 0 else 0 
recall = round((vc / (vc + vw))*100, 2) if (vc + vw) > 0 else 0
f1 = round(2* precision* recall/ (precision + recall), 2) if (precision + recall) > 0 else 0

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1: {f1}")

Precision: 98.82
Recall: 99.05
F1: 98.93
